# Hackaton Bernas da Street

In [2]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from IPython.display import Image
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.base import BaseEstimator, ClassifierMixin, TransformerMixin
from sklearn.impute import SimpleImputer

In [4]:
df = pd.read_csv("data/train.csv")
df.head()

,id,timestamp,product_id,product_department,product_category,card_id,user_id,C15,C16,C17,C18,C19,C20,C21,amount,isfraud
0,202,1413851368636,fa41b2d8,ef5ed6b8,50e219e0,ecad2386,bd544220,300,50,2312,0,167,100075,16,195.65,0
1,9909,1413851725946,fa41b2d8,ef5ed6b8,50e219e0,ecad2386,bd544220,300,50,2312,0,167,100075,16,195.65,0
2,22692,1413851873924,fa41b2d8,ef5ed6b8,50e219e0,ecad2386,bd544220,300,50,2312,0,167,100075,16,195.65,0
3,22938,1413851651534,fa41b2d8,ef5ed6b8,50e219e0,ecad2386,bd544220,300,50,2312,0,167,100075,16,195.65,0
4,29187,1413853025188,fa41b2d8,ef5ed6b8,50e219e0,ecad2386,bd544220,300,50,2312,0,167,100075,16,195.65,0


# Data Cleaning 101

In [5]:
df.describe()

,id,timestamp,C15,C16,C17,C18,C19,C20,C21,amount,isfraud
count,5.224120e+05,5.224120e+05,522412.000000,522412.000000,522412.000000,522412.000000,522412.000000,522412.000000,522412.000000,522412.000000,522412.000000
mean,1.669530e+07,1.414202e+12,320.588156,52.683499,1660.976921,1.788004,427.071648,43592.443851,72.838093,189.437083,0.105717
std,9.127556e+06,2.084992e+08,28.896147,30.344078,1016.566892,1.366340,517.917622,49633.190195,69.107663,11.654796,0.307476
min,2.020000e+02,1.413850e+12,120.000000,20.000000,112.000000,0.000000,33.000000,-1.000000,13.000000,52.960000,0.000000
25%,1.036983e+07,1.414042e+12,320.000000,50.000000,423.000000,0.000000,35.000000,-1.000000,20.000000,191.770000,0.000000
50%,1.417443e+07,1.414132e+12,320.000000,50.000000,2227.000000,3.000000,163.000000,-1.000000,48.000000,191.770000,0.000000
75%,2.540063e+07,1.414412e+12,320.000000,50.000000,2506.000000,3.000000,687.000000,100083.000000,95.000000,191.770000,0.000000
max,3.237738e+07,1.414541e+12,1024.000000,1024.000000,2688.000000,3.000000,1839.000000,100248.000000,221.000000,229.450000,1.000000


In [6]:
df.isnull().sum()

id                    0
timestamp             0
product_id            0
product_department    0
product_category      0
card_id               0
user_id               0
C15                   0
C16                   0
C17                   0
C18                   0
C19                   0
C20                   0
C21                   0
amount                0
isfraud               0
dtype: int64

In [15]:
df.nunique()

id                    522412
timestamp             521650
product_id              1959
product_department      1251
product_category           4
card_id                 1668
user_id                82591
C15                        8
C16                        9
C17                      293
C18                        4
C19                       56
C20                      157
C21                       50
amount                  1187
isfraud                    2
dtype: int64

In [13]:
df.card_id.value_counts()

f53417e1    162255
ecad2386     63340
7358e05e     25178
d292c32f     20410
e2fcccd2     18810
3692fd30     13678
03a08c3f      9405
54c5d545      9057
5064623a      8635
e71aba61      7813
73206397      6930
e2a1ca37      6906
03528b27      6630
06ef0828      6510
66f5e02e      4436
6a7bc718      4409
f888bf4c      4270
be7c618d      4248
39947756      4078
460f4063      3816
c0c5eb49      3557
4e02fbd3      3529
c680d3dd      3350
422da614      3333
1dc72b4d      2805
170f5944      2798
2bc6bb64      2717
fb7c70a3      2242
75759a0b      2217
1d64ced5      2187
             ...  
14910e7d         1
424e1cf5         1
a7e77eb3         1
c02a0551         1
4b2de55b         1
1d81d943         1
1c9b2dd5         1
76996387         1
7f063018         1
064bead1         1
5b005139         1
f3bd35bd         1
ebc30132         1
423f3c5b         1
294c034a         1
9d225a01         1
d6004e51         1
eac7f9fe         1
cd95919d         1
854ececa         1
c9f8e63a         1
997b15df    

In [17]:
df = df.drop("id", axis=1)

In [20]:
df.head()

,timestamp,product_id,product_department,product_category,card_id,user_id,C15,C16,C17,C18,C19,C20,C21,amount,isfraud
0,1413851368636,fa41b2d8,ef5ed6b8,50e219e0,ecad2386,bd544220,300,50,2312,0,167,100075,16,195.65,0
1,1413851725946,fa41b2d8,ef5ed6b8,50e219e0,ecad2386,bd544220,300,50,2312,0,167,100075,16,195.65,0
2,1413851873924,fa41b2d8,ef5ed6b8,50e219e0,ecad2386,bd544220,300,50,2312,0,167,100075,16,195.65,0
3,1413851651534,fa41b2d8,ef5ed6b8,50e219e0,ecad2386,bd544220,300,50,2312,0,167,100075,16,195.65,0
4,1413853025188,fa41b2d8,ef5ed6b8,50e219e0,ecad2386,bd544220,300,50,2312,0,167,100075,16,195.65,0


## Card id investigation

## Sort timestamp and split test/train

In [25]:
df = df.sort_values(by = "timestamp").reset_index(drop = True)
df.head()

,timestamp,product_id,product_department,product_category,card_id,user_id,C15,C16,C17,C18,C19,C20,C21,amount,isfraud
0,1413849604595,c4e18dd6,85f751fd,50e219e0,92e72531,a99f214a,320,50,2480,3,297,100111,61,191.77,0
1,1413849611766,c4e18dd6,85f751fd,50e219e0,e71aba61,a99f214a,320,50,1722,0,35,-1,79,191.77,1
2,1413849613367,dd7026ee,15d93b0b,50e219e0,ecad2386,5c7c1b02,320,50,2495,2,167,100173,23,227.63,0
3,1413849619068,c4e18dd6,85f751fd,50e219e0,5e3f096f,ba2d210a,320,50,2161,0,35,100051,157,191.77,0
4,1413849625209,c4e18dd6,85f751fd,50e219e0,39947756,0ddad6d9,320,50,1955,3,163,100192,71,191.77,0


In [31]:
len(df)

417929

In [28]:
len(df.iloc[0:int(0.8*len(df))])

417929

In [29]:
len(df.iloc[int(0.8*len(df)):])

104483

In [41]:
df_train = df.iloc[0:int(0.8*len(df))].copy()
df_test = df.iloc[int(0.8*len(df)):].copy()
x_train = df_train.drop("isfraud",axis=1)
y_train = df_train[["isfraud"]]
x_test = df_test.drop("isfraud",axis=1)
y_test = df_test[["isfraud"]]

# Baseline Basic Model for Dummies

In [45]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(x_train[["amount"]], y_train)

/anaconda3/envs/hckt01/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/envs/hckt01/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
lr.score(, y_ful)

# 